In [18]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import warnings;
warnings.filterwarnings("ignore")
import re
#pd.set_option("display.max_columns",None)
#pd.set_option("display.max_rows",None)

In [19]:
df = pd.read_csv("cleaned_step1.csv")

### 1) Body Type

In [20]:
df["body_type"].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [21]:
df["body_type"].fillna(df.groupby('make_model')['body_type'].transform(lambda x: x.mode()[0]), inplace=True)

### 2.0 Type

In [22]:
df['Type'].value_counts(dropna=False)

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: Type, dtype: int64

In [23]:
df['Type'] = df['Type'].fillna('Used')

### 2) Year

In [24]:
df["registration"].value_counts(dropna=False)

2018.0    4522
2016.0    3674
2017.0    3273
2019.0    2853
NaN       1597
Name: registration, dtype: int64

In [25]:
df['registration']=df['registration'].fillna(df.groupby('Type')['registration'].transform(lambda x:x.mode()[0]))

In [26]:
df['registration'] = df['registration'].fillna(2016)

### 3) VAT

In [27]:
df.vat.value_counts(dropna=False)

VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64

In [28]:
df['vat'] = df['vat'].fillna('VAT not deductible')

In [29]:
df["vat"] = df["vat"].replace("VAT not deductible","VAT undeductible")

In [30]:
df["vat"].value_counts(dropna=False)

VAT deductible      10980
VAT undeductible     4513
Price negotiable      426
Name: vat, dtype: int64

### 4) KM

In [31]:
df["km"].isnull().sum()

1024

In [32]:
df["km"].fillna(df.groupby(['registration','Displacement']).km.transform(lambda x: x.mean()), inplace=True)

ValueError: Length mismatch: Expected axis has 15423 elements, new values have 15919 elements

In [33]:
mean_group_km1=df.groupby(['registration']).km.transform(lambda x: x.mean())

In [34]:
df["km"].fillna(mean_group_km1, inplace=True)

### 5) Previous Owner

In [19]:
index=df[df["km"]<10].index

In [20]:
df.loc[index,"prev_owner"]=0.0

In [21]:
df.loc[(df["Type"]=="New") & (df["prev_owner"].isnull()),"prev_owner"]=1.0

In [22]:
df.loc[((df["registration"]==2019) | (df["km"]<5000)) & (df["prev_owner"].isnull()),"prev_owner"]=1.0

In [23]:
mode_group_po = df.groupby(['registration'])['prev_owner'].transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [24]:
df["prev_owner"].fillna(mode_group_po, inplace=True)

### hp 

In [25]:
df['hp'].value_counts(dropna=False)

85.0     2542
66.0     2122
81.0     1402
100.0    1308
110.0    1112
         ... 
75.0        1
115.0       1
137.0       1
132.0       1
84.0        1
Name: hp, Length: 81, dtype: int64

In [26]:
df[df.make_model == 'Opel Astra'][['body_type','hp', 'Displacement']].head(30)

,body_type,hp,Displacement
5712,Station wagon,100.0,1598.0
5713,Station wagon,100.0,NaN
5714,Station wagon,100.0,1598.0
5715,Station wagon,100.0,1598.0
5716,Station wagon,100.0,NaN
5717,Compact,100.0,1598.0
5718,Station wagon,100.0,NaN
5719,Station wagon,100.0,1600.0
5720,Station wagon,100.0,1598.0
5721,Station wagon,100.0,1598.0


In [27]:
mode_hp = df[~(df['Displacement'].isnull())]\
              .groupby(['make_model','body_type','Displacement'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [28]:
df.loc[~(df['Displacement'].isnull()),"hp"] = df.loc[~(df['Displacement'].isnull()),"hp"].fillna(mode_hp)

In [29]:
mode_hp1 = df[~(df['Cylinders'].isnull())]\
              .groupby(['make_model','body_type','Cylinders'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [30]:
df.loc[~(df['Cylinders'].isnull()),"hp"] = df.loc[~(df['Cylinders'].isnull()),"hp"].fillna(mode_hp1)

In [31]:
mode_hp2 = df.groupby(['make_model','body_type'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [32]:
df['hp'] = df['hp'].fillna(mode_hp2)

In [33]:
df['hp'].isnull().sum()

0

 ### 7) Type

In [ ]:
df['Type'].value_counts(dropna=False)

In [ ]:
df['Type'] = df['Type'].fillna('Used')

### 8) Next Inspection

In [34]:
df['Next Inspection'].value_counts(dropna=False)

NaN           12384
2021-06-01      471
2021-03-01      210
2021-05-01      180
2021-04-01      171
              ...  
2017-12-01        1
2018-01-01        1
2017-02-01        1
2016-05-01        1
2024-03-01        1
Name: Next Inspection, Length: 78, dtype: int64

In [35]:
df.drop(["Next Inspection"],axis=1,inplace=True)

### 9) Inspection New

In [36]:
df['Inspection new'].value_counts(dropna=False)

NaN    11987
Yes     3932
Name: Inspection new, dtype: int64

In [37]:
df['Inspection new'] = df['Inspection new'].fillna('No')

### 10) Warranty(months)

In [38]:
df['Warranty(months)'].value_counts(dropna=False)

NaN     11066
12.0     2594
24.0     1118
60.0      401
36.0      279
48.0      149
6.0       125
72.0       59
3.0        33
23.0       11
18.0       10
20.0        7
25.0        6
2.0         5
26.0        4
50.0        4
16.0        4
1.0         3
4.0         3
34.0        3
19.0        3
13.0        3
11.0        2
21.0        2
14.0        2
17.0        2
45.0        2
46.0        2
9.0         2
22.0        2
28.0        2
56.0        1
47.0        1
10.0        1
8.0         1
7.0         1
40.0        1
33.0        1
65.0        1
15.0        1
30.0        1
49.0        1
Name: Warranty(months), dtype: int64

In [39]:
df['Warranty(months)'] = df['Warranty(months)'].fillna(0.0)

### 11) Body Color

In [40]:
df['Body Color'].value_counts(dropna=False)

Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: Body Color, dtype: int64

In [41]:
mode_color = df.groupby(['make_model','body_type'])['Body Color']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [42]:
df['Body Color'] = df['Body Color'].fillna(mode_color)

In [43]:
mode_color1 = df.groupby(['make_model'])['Body Color']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [44]:
df['Body Color'] = df['Body Color'].fillna(mode_color1)

### 11) Paint Type

In [45]:
df['Paint Type'].value_counts(dropna=False)

Metallic       9794
NaN            5772
Uni/basic       347
Perl effect       6
Name: Paint Type, dtype: int64

In [46]:
mode_paint = df.groupby(['make_model','body_type'])['Paint Type']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [47]:
df['Paint Type'] = df['Paint Type'].fillna(mode_paint)

In [48]:
mode_paint1 = df.groupby(['make_model'])['Paint Type']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [49]:
df['Paint Type'] = df['Paint Type'].fillna(mode_paint1)

### 13) Nr. of Doors

In [50]:
df['Nr. of Doors'].value_counts(dropna=False)

5.0    11575
4.0     3079
3.0      832
2.0      219
NaN      212
7.0        1
1.0        1
Name: Nr. of Doors, dtype: int64

In [51]:
mode_nr_doors = df.groupby(['make_model','body_type'])['Nr. of Doors']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [52]:
df['Nr. of Doors'] = df['Nr. of Doors'].fillna(mode_nr_doors)

In [53]:
mode_nr_doors1 = df.groupby(['make_model'])['Nr. of Doors']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [54]:
df['Nr. of Doors'] = df['Nr. of Doors'].fillna(mode_nr_doors1)

### 14) Nr. of Seats

In [55]:
df['Nr. of Seats'].value_counts(dropna=False)

5.0    13336
4.0     1125
NaN      977
7.0      362
2.0      116
6.0        2
3.0        1
Name: Nr. of Seats, dtype: int64

In [56]:
mode_nr_seats = df.groupby(['make_model','body_type'])['Nr. of Seats']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [57]:
df['Nr. of Seats'] = df['Nr. of Seats'].fillna(mode_nr_seats)

In [58]:
mode_nr_seats1 = df.groupby(['make_model'])['Nr. of Seats']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [59]:
df['Nr. of Seats'] = df['Nr. of Seats'].fillna(mode_nr_seats1)

### 15) Gearing Type

In [60]:
df['Gearing Type'].value_counts(dropna=False)

Manual            8153
Automatic         7297
Semi-automatic     469
Name: Gearing Type, dtype: int64

### 16) Displacement

In [61]:
df['Displacement'].value_counts(dropna=False)

1598.0    4761
999.0     2438
1398.0    1314
1399.0     749
1229.0     677
          ... 
2967.0       1
1390.0       1
140.0        1
1350.0       1
1198.0       1
Name: Displacement, Length: 78, dtype: int64

In [62]:
df['Displacement'].isnull().sum()

496

In [63]:
mode_displacement = df.groupby(['make_model','body_type','hp'])['Displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [64]:
df['Displacement'] = df['Displacement'].fillna(mode_displacement)

In [65]:
mode_displacement1 = df.groupby(['make_model','body_type'])['Displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [66]:
df['Displacement'] = df['Displacement'].fillna(mode_displacement1)

In [67]:
mode_displacement2 = df.groupby(['make_model'])['Displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [68]:
df['Displacement'] = df['Displacement'].fillna(mode_displacement2)

### 16) Cylinders

In [72]:
df['Cylinders'].value_counts(dropna=False)

4.0    8105
NaN    5680
3.0    2104
5.0      22
6.0       3
2.0       2
8.0       2
1.0       1
Name: Cylinders, dtype: int64

In [73]:
mode_cylinders = df.groupby(['make_model', 'Displacement'])['Cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [74]:
df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders)

In [125]:
mode_cylinders1 = df.groupby(['make_model', 'Weight(kg)'])['Cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [126]:
df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders1)

In [127]:
mode_cylinders2 = df.groupby(['make_model'])['Cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [128]:
df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders2)

In [129]:
df['Cylinders'] = df['Cylinders'].fillna(4.0)

### 17) Weight(kg)

In [77]:
df.columns

Index(['make_model', 'body_type', 'price', 'vat', 'km', 'registration',
       'prev_owner', 'hp', 'Type', 'Inspection new', 'Warranty(months)',
       'Body Color', 'Paint Type', 'Upholstery', 'Nr. of Doors',
       'Nr. of Seats', 'Gearing Type', 'Displacement', 'Cylinders',
       'Weight(kg)', 'Drive chain', 'Fuel', 'CO2 Emission(g CO2/km)',
       'Emission Class', 'Comfort&Convenience', 'Entertainment&Media',
       'Extras', 'Safety & Security', 'description', 'Gears',
       'upholstery_material', 'Consumption_combined', 'Consumption_city',
       'Consumption_country'],
      dtype='object')

In [78]:
df['Weight(kg)'].value_counts(dropna=False)

NaN       6974
1163.0     574
1360.0     356
1165.0     301
1335.0     242
          ... 
900.0        1
1650.0       1
1428.0       1
2355.0       1
1523.0       1
Name: Weight(kg), Length: 435, dtype: int64

In [79]:
mode_weight = df.groupby(['make_model', 'Displacement'])['Weight(kg)']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [80]:
df['Weight(kg)'] = df['Weight(kg)'].fillna(mode_weight)

In [81]:
mode_weight1 = df.groupby(['make_model'])['Weight(kg)']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [82]:
df['Weight(kg)'] = df['Weight(kg)'].fillna(mode_weight1)

In [83]:
mode_weight2 = df.groupby(['make_model', 'hp'])['Weight(kg)']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [84]:
df['Weight(kg)'] = df['Weight(kg)'].fillna(mode_weight2)

In [85]:
mode_weight3 = df.groupby(['body_type'])['Weight(kg)']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [86]:
df['Weight(kg)'] = df['Weight(kg)'].fillna(mode_weight3)

### 18) Drive Chain

In [87]:
df['Drive chain'].value_counts(dropna=False)

front    8886
NaN      6858
4WD       171
rear        4
Name: Drive chain, dtype: int64

In [88]:
mode_drive_chain = df.groupby(['make_model', 'body_type'])['Drive chain']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [89]:
df['Drive chain'] = df['Drive chain'].fillna(mode_drive_chain)

In [90]:
mode_drive_chain1 = df.groupby(['make_model'])['Drive chain']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [91]:
df['Drive chain'] = df['Drive chain'].fillna(mode_drive_chain1)

### 19) Fuel

In [92]:
df['Fuel'].value_counts(dropna=False)

Benzine     8549
Diesel      7299
Gas           64
Others         6
Electric       1
Name: Fuel, dtype: int64

### 20) CO2 Emission(g CO2/km)

In [93]:
df['CO2 Emission(g CO2/km)'].value_counts(dropna=False)

NaN      2436
120.0     740
99.0      545
97.0      537
104.0     501
         ... 
193.0       1
239.0       1
51.0        1
45.0        1
253.0       1
Name: CO2 Emission(g CO2/km), Length: 120, dtype: int64

In [94]:
mode_CO2 = df.groupby(['make_model','Displacement'])['CO2 Emission(g CO2/km)']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [95]:
df['CO2 Emission(g CO2/km)'] = df['CO2 Emission(g CO2/km)'].fillna(mode_CO2)

In [96]:
mode_CO2_1 = df.groupby(['make_model'])['CO2 Emission(g CO2/km)']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [97]:
df['CO2 Emission(g CO2/km)'] = df['CO2 Emission(g CO2/km)'].fillna(mode_CO2_1)

In [98]:
mode_CO2_2 = df.groupby(['body_type'])['CO2 Emission(g CO2/km)']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [99]:
df['CO2 Emission(g CO2/km)'] = df['CO2 Emission(g CO2/km)'].fillna(mode_CO2_2)

### 21) Emission Class

In [100]:
df['Emission Class'].value_counts(dropna=False)

Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: Emission Class, dtype: int64

In [101]:
mode_Emision_C= df.groupby(['CO2 Emission(g CO2/km)'])['Emission Class']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [102]:
df['Emission Class'] = df['Emission Class'].fillna(mode_Emision_C)

In [103]:
mode_Emision_C1= df.groupby(['registration'])['Emission Class']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [104]:
df['Emission Class'] = df['Emission Class'].fillna(mode_Emision_C1)

### 22) Gears

In [105]:
df['Gears'].value_counts(dropna=False)

6.0    5822
NaN    4712
5.0    3240
7.0    1908
8.0     224
9.0       6
4.0       2
3.0       2
1.0       2
2.0       1
Name: Gears, dtype: int64

In [106]:
mode_Gears= df.groupby(['make_model','Gearing Type'])['Gears']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [107]:
df['Gears'] = df['Gears'].fillna(mode_Gears)

In [108]:
mode_Gears1= df.groupby(['Gearing Type'])['Gears']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [109]:
df['Gears'] = df['Gears'].fillna(mode_Gears1)

### 24) Upholstery Material

In [110]:
df['upholstery_material'].value_counts(dropna=False)

Cloth, Black           5821
NaN                    4503
Part leather, Black    1121
Cloth                  1005
Cloth, Grey             891
Cloth, Other            639
Full leather, Black     575
Other, Other            182
Part leather            140
Full leather            139
Full leather, Brown     116
Part leather, Grey      116
Other, Black            110
Full leather, Other      72
Full leather, Grey       67
Part leather, Other      65
Other                    56
Part leather, Brown      50
alcantara, Black         47
Full leather, Beige      36
Velour, Black            36
Cloth, Brown             28
Velour                   16
Other, Grey              15
Cloth, Beige             13
Cloth, Blue              12
Cloth, White              8
Velour, Grey              8
alcantara, Grey           6
Cloth, Red                5
Other, Yellow             4
Part leather, Red         3
Part leather, White       2
Part leather, Beige       2
alcantara                 2
Full leather, Blue  

In [123]:
df.drop('upholstery_material', axis = 1, inplace = True)

# 26) Consumption Combined

In [111]:
df['Consumption_combined'].value_counts(dropna=False)

NaN     2033
5.4      770
3.9      733
4.0      713
5.1      657
        ... 
43.0       1
54.0       1
13.0       1
11.0       1
46.0       1
Name: Consumption_combined, Length: 73, dtype: int64

In [112]:
coms = ((df['Consumption_city']+df['Consumption_country'])/2)

In [113]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(coms)

In [114]:
mode_Cons= df.groupby(['CO2 Emission(g CO2/km)'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [115]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons)

In [116]:
mode_Cons1= df.groupby(['Displacement'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [117]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons1)

In [118]:
mode_Cons2= df.groupby(['make_model'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [119]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons2)

In [120]:
df['Consumption_combined'].isnull().sum()

0

In [121]:
df.columns

Index(['make_model', 'body_type', 'price', 'vat', 'km', 'registration',
       'prev_owner', 'hp', 'Type', 'Inspection new', 'Warranty(months)',
       'Body Color', 'Paint Type', 'Upholstery', 'Nr. of Doors',
       'Nr. of Seats', 'Gearing Type', 'Displacement', 'Cylinders',
       'Weight(kg)', 'Drive chain', 'Fuel', 'CO2 Emission(g CO2/km)',
       'Emission Class', 'Comfort&Convenience', 'Entertainment&Media',
       'Extras', 'Safety & Security', 'description', 'Gears',
       'upholstery_material', 'Consumption_combined', 'Consumption_city',
       'Consumption_country'],
      dtype='object')

In [122]:
df.corr()['Weight(kg)'].sort_values()

registration             -0.081018
prev_owner                0.018077
Consumption_city          0.039197
Warranty(months)          0.068275
Consumption_country       0.076362
Consumption_combined      0.087133
Nr. of Doors              0.149248
km                        0.167781
Nr. of Seats              0.313700
CO2 Emission(g CO2/km)    0.325058
Cylinders                 0.333504
Gears                     0.346933
price                     0.453318
Displacement              0.531121
hp                        0.655865
Weight(kg)                1.000000
Name: Weight(kg), dtype: float64

In [131]:
df.drop(['Upholstery', 'Consumption_city', 'Consumption_country'], axis = 1, inplace = True)

In [132]:
df.isnull().sum()

make_model                   0
body_type                    0
price                        0
vat                          0
km                           0
registration                 0
prev_owner                   0
hp                           0
Type                         0
Inspection new               0
Warranty(months)             0
Body Color                   0
Paint Type                   0
Nr. of Doors                 0
Nr. of Seats                 0
Gearing Type                 0
Displacement                 0
Cylinders                    0
Weight(kg)                   0
Drive chain                  0
Fuel                         0
CO2 Emission(g CO2/km)       0
Emission Class               0
Comfort&Convenience        920
Entertainment&Media       1374
Extras                    2962
Safety & Security          982
description                  0
Gears                        0
Consumption_combined         0
dtype: int64

In [133]:
df.to_csv("missings_filled_step2.csv", index=False)